In [1]:
#@title Import libraries {display-mode: "form"}

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

pd.options.display.max_columns = None

sns.set()

pd.options.mode.chained_assignment = None

from sklearn.model_selection import train_test_split # Data splitting for modeling
from sklearn.metrics import mean_absolute_error as mae # Error metric for models

In [2]:
# MLB_data_2013_2020 = pd.read_excel(open('MLB Games 2013-19.xlsx', 'rb'), sheet_name='Training')

In [3]:
# MLB_data_2013_2020 = pd.read_excel(open('MLB Games 2013-19-Copy1.xlsx', 'rb'), sheet_name='Training')

In [4]:
# MLB_data_2013_2020 = MLB_data_2013_2020.drop(cols = ['away_Game_Scores', 'away_RISP', 'home_Strikes', 
# 'home_Game_Scores', 'Venue', 'home_RISP', 'ID', 
#  'PC_home', 'ST_home', 'away_FirstPitch', 'TB_home', 'TB_away', 'away_ip', 'ST_away', 
#  'game_id', 'field_type', 'game_time', 'home_ip', 'PC_away', 'away_Strikes', 
#  'away_Balls', 'home_Balls', 'home_FirstPitch'])

In [5]:
# MLB_data_2013_2020.columns

In [6]:
# MLB_data_2013_2020 = MLB_data_2013_2020[df_final.columns]

In [7]:
# MLB_data_1998_2012 = pd.read_excel(open('MLB Games 1998-2012.xlsx', 'rb'), sheet_name='Training')

#Renaming to match the other table
# MLB_data_1998_2012 = MLB_data_1998_2012.rename(columns=
#                                                {"Year": "year", "Game_No": "game_no",
#                                                 "PO_away": "putouts_away", "SO_away": "k_away",
#                                                 "away_IP": "away_ip", "A_away": "assists_away",
#                                                 "away_GameScores": "away_Game_Scores",
#                                                 "PO_home": "putouts_home", "SO_away": "k_away",
#                                                 "home_IP": "home_ip", "A_home": "assists_home",
#                                                "home_GameScores": "home_Game_Scores"})

In [8]:
# MLB_data = pd.concat([MLB_data_1998_2012, MLB_data_2013_2020], axis=0)
MLB_data = pd.read_csv('mlb_games-Copy1.csv')
MLB_data = MLB_data.iloc[:, 1:]
# MLB_data.info()

In [9]:
MLB_data['game_date'] = pd.to_datetime(MLB_data['game_date'], utc=False)
# MLB_data['game_duration'] = MLB_data['game_duration'].astype('int64')
# MLB_data['putouts_away'] = MLB_data['putouts_away'].astype('int64')
# MLB_data['putouts_home'] = MLB_data['putouts_away'].astype('int64')

In [10]:
# Create win/lose label
MLB_data['Winner'] = MLB_data.apply(lambda x: x['Home_team_full'] \
                                    if x['score_home'] > x['score_away'] \
                                    else x['Away_team_full'], axis = 1)
MLB_data['Loser'] = MLB_data.apply(lambda x: x['Home_team_full'] \
                                   if x['score_home'] < x['score_away'] \
                                   else x['Away_team_full'], axis = 1)

In [11]:
# MLB_data.reset_index()

In [12]:
MLB_data['Away_team_full'].unique()

array(['Kansas City Royals', 'Cleveland Indians', 'Detroit Tigers',
       'Chicago White Sox', 'Colorado Rockies', 'Milwaukee Brewers',
       'San Diego Padres', 'Chicago Cubs', 'San Francisco Giants',
       'Philadelphia Phillies', 'Los Angeles Dodgers', 'New York Yankees',
       'Boston Red Sox', 'Minnesota Twins', 'Pittsburgh Pirates',
       'Texas Rangers', 'Washington Nationals', 'Tampa Bay Rays',
       'Baltimore Orioles', 'Toronto Blue Jays', 'New York Mets',
       'St. Louis Cardinals', "Arizona D'Backs", 'Miami Marlins',
       'Atlanta Braves', 'Cincinnati Reds', 'Houston Astros',
       'Seattle Mariners', 'Los Angeles Angels', 'Oakland Athletics'],
      dtype=object)

In [13]:
# MLB_data = MLB_data.drop(columns = 
#                          ['ID', \
#                           'away_ip', 'away_Game_Scores', 'away_RISP', \
#                           'away_Balls', 'away_Strikes', 'away_FirstPitch', \
#                           'home_ip', 'home_Game_Scores', 'home_RISP', \
#                           'home_Balls', 'home_Strikes', 'home_FirstPitch'])

## Dataframe Version

In [14]:
#get annual vectors for each team
def getAnnualTeamData3(teamName, year):
    
    annual_data = MLB_data[MLB_data['year'] == year]

    # number of games home and away
    gamesHome = annual_data[annual_data['Home_team_full'] == teamName] 
    gamesAway = annual_data[annual_data['Away_team_full'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)

    # runs per game
    totalRunsScored = gamesHome['score_home'].sum()
    totalRunsScored += gamesAway['score_away'].sum()
    totalRunsAllowed = gamesHome['score_away'].sum()
    totalRunsAllowed += gamesAway['score_home'].sum()
    
    #total AtBats
    totalAB = gamesHome['AB_home'].sum()
    totalAB += gamesAway['AB_away'].sum()
    totalABAllowed = gamesHome['AB_away'].sum()
    totalABAllowed += gamesAway['AB_home'].sum()

    #total Hits
    totalH = gamesHome['H_home'].sum()
    totalH += gamesAway['H_away'].sum()
    totalHAllowed = gamesHome['H_away'].sum()
    totalHAllowed += gamesAway['H_home'].sum()
    
    #total E
    totalE = gamesHome['E_home'].sum()
    totalE += gamesAway['E_away'].sum()
    
    #total 2B (doubles)
    total2B = gamesHome['2B_home'].sum()
    total2B += gamesAway['2B_away'].sum()
    total2BAllowed = gamesHome['2B_away'].sum()
    total2BAllowed += gamesAway['2B_home'].sum()
    
    #total 3B (triples)
    total3B = gamesHome['3B_home'].sum()
    total3B += gamesAway['3B_away'].sum()
    total3BAllowed = gamesHome['3B_away'].sum()
    total3BAllowed += gamesAway['3B_home'].sum()
    
    # total HR (home runs)
    totalHR = gamesHome['HR_home'].sum()
    totalHR += gamesAway['HR_away'].sum()
    totalHRAllowed = gamesHome['HR_away'].sum()
    totalHRAllowed += gamesAway['HR_home'].sum()

    # total RBI (run-batted in)
    totalRBI = gamesHome['RBI_home'].sum()
    totalRBI += gamesAway['RBI_away'].sum()

    # total SF (sac flies)
    totalSH = gamesHome['SH_home'].sum()
    totalSH += gamesAway['SH_away'].sum()
    
    # total SF (sac flies)
    totalSF = gamesHome['SF_home'].sum()
    totalSF += gamesAway['SF_away'].sum()
    totalSFAllowed = gamesHome['SF_away'].sum()
    totalSFAllowed += gamesAway['SF_home'].sum()

    # total HBP (hit by pitches)
    totalHBP = gamesHome['HBP_home'].sum()
    totalHBP += gamesAway['HBP_away'].sum()  
    totalHBPAllowed = gamesHome['HBP_away'].sum()
    totalHBPAllowed += gamesAway['HBP_home'].sum()
    
    # total BB
    totalBB = gamesHome['BB_home'].sum()
    totalBB += gamesAway['BB_away'].sum()
    totalBBAllowed = gamesHome['BB_away'].sum()
    totalBBAllowed += gamesAway['BB_home'].sum()
    
    # total IBB
    totalIBB = gamesHome['IBB_home'].sum()
    totalIBB += gamesAway['IBB_away'].sum()

    # total k
    totalSO = gamesHome['k_home'].sum()
    totalSO += gamesAway['k_away'].sum()

    # total teamER
    totalteamER = gamesHome['teamER_home'].sum()
    totalteamER += gamesAway['teamER_away'].sum()

    # total putouts
    totalputouts = gamesHome['putouts_home'].sum()
    totalputouts += gamesAway['putouts_away'].sum()

    #Batting stats
    gamesWonPercentage = numGamesWon / numGames
    RunsPerGame = totalRunsScored / numGames
    
    RunsDifference = totalRunsScored - totalRunsAllowed
    
    BA = totalH / totalAB
    OBP = (totalH + totalBB + totalHBP) / (totalAB + totalBB + totalHBP + totalSF)
    total1B = totalH - total2B - total3B - totalHR
    SLG = (1*total1B + 2*total2B + 3*total3B + 4*totalHR) / (totalAB)
    
    OBA = totalHAllowed / totalABAllowed
    OOBP = (totalHAllowed + totalBBAllowed + totalHBPAllowed) \
    / (totalABAllowed + totalBBAllowed + totalHBPAllowed + totalSFAllowed)
    total1BAllowed = totalHAllowed - total2BAllowed - total3BAllowed - totalHRAllowed
    OSLG = (1*total1BAllowed + 2*total2BAllowed + 3*total3BAllowed + 4*totalHRAllowed) / (totalABAllowed)
    
    #Defense stats
    totalPA = totalAB + totalBB + totalHBP + totalSF + totalSH
    RunsAllowedPerGame = totalRunsAllowed / numGames
    DefEff = 1 - ((totalH + totalE - totalHR)/(totalPA - totalBB - totalSO - totalHBP - totalHR))
    
    #Pitching stats
    ERA = (totalteamER / (totalputouts / 3)) * 9
    
    result_dict = {'Team': [teamName], 'W': [int(numGamesWon)], 'L': [int(numGamesLost)],
                   'W-L%': [round(gamesWonPercentage, 3)],
                   'RS': [int(totalRunsScored)],
                   'R/G': [round(RunsPerGame, 2)], \
                   'RA': [int(totalRunsAllowed)],
                   'RA/G': [round(RunsAllowedPerGame, 2)], \
                   'RD': [int(RunsDifference)],
                   'AB': [int(totalAB)], 'H': [int(totalH)], \
                   '2B': [int(total2B)], '3B': [int(total3B)], \
                   'HR': [int(totalHR)], 'RBI': [int(totalRBI)], \
                   'BB': [int(totalBB)], 'IBB': [int(totalIBB)], \
                   'BA': [round(BA, 3)], 'OBP': [round(OBP, 3)], \
                   'SLG': [round(SLG, 3)], \
                   'DefEff': [round(DefEff, 3)], \
                   'SO': [int(totalSO)], 'ERA': [round(ERA, 2)], \
                   'OBA': [round(OBA, 3)],
                   'OOBP': [round(OOBP, 3)], \
                   'OSLG': [round(OSLG, 3)]}
    result_team = pd.DataFrame.from_dict(result_dict)
    return result_team

In [15]:
import collections

def createAnnualDict3(year):
    annualDictionary = collections.defaultdict(list)
    teamList = MLB_data['Away_team_full'].unique()
    column_names = ['Team', 'W', 'L', 'W-L%', 
                    'RS', 'R/G', 'RA', 'RA/G', 'RD', 'AB', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
                    'IBB', 'BA', 'OBP', 'SLG', 'DefEff', 'SO', 'ERA', 'OBA', 'OOBP', 'OSLG']

    result_df = pd.DataFrame(columns = column_names)

    for team in teamList:
        team_vector = getAnnualTeamData3(team, year)
        result_df = pd.concat([result_df, team_vector])
#         annualDictionary[team] = team_vector
    return result_df

In [16]:
createAnnualDict3(1999).sort_values(by='Team', ascending=True).reset_index().drop(columns = 'index')

,Team,W,L,W-L%,RS,R/G,RA,RA/G,RD,AB,H,2B,3B,HR,RBI,BB,IBB,BA,OBP,SLG,DefEff,SO,ERA,OBA,OOBP,OSLG
0,Arizona D'Backs,100,62,0.617,908,5.60,676,4.17,232,5658,1566,289,46,216,865,588,52,0.277,0.347,0.459,0.678,1045,3.77,0.249,0.320,0.402
1,Atlanta Braves,103,59,0.636,840,5.19,661,4.08,179,5569,1481,309,23,197,791,608,62,0.266,0.341,0.436,0.692,962,3.63,0.251,0.314,0.377
2,Baltimore Orioles,78,84,0.481,851,5.25,815,5.03,36,5637,1572,299,21,203,804,615,34,0.279,0.353,0.447,0.686,890,4.77,0.269,0.348,0.439
3,Boston Red Sox,94,68,0.580,836,5.16,718,4.43,118,5579,1551,334,42,176,808,597,27,0.278,0.350,0.448,0.671,928,4.00,0.253,0.315,0.398
4,Chicago Cubs,67,95,0.414,747,4.61,920,5.68,-173,5482,1411,255,35,189,717,571,38,0.257,0.329,0.420,0.678,1170,5.27,0.286,0.346,0.467
5,Chicago White Sox,75,86,0.463,777,4.80,870,5.37,-93,5644,1563,298,37,162,742,499,22,0.277,0.337,0.429,0.677,810,4.92,0.282,0.353,0.447
6,Cincinnati Reds,96,67,0.589,865,5.31,711,4.36,154,5649,1536,312,37,209,820,569,37,0.272,0.341,0.451,0.677,1125,3.98,0.241,0.324,0.411
7,Cleveland Indians,97,65,0.599,1009,6.23,860,5.31,149,5634,1629,309,32,209,960,743,41,0.289,0.373,0.467,0.657,1099,4.89,0.268,0.346,0.438
8,Colorado Rockies,72,90,0.444,906,5.59,1028,6.35,-122,5717,1644,305,39,223,863,508,31,0.288,0.348,0.472,0.675,863,6.01,0.301,0.384,0.499
9,Detroit Tigers,69,92,0.429,747,4.64,882,5.48,-135,5481,1433,289,34,212,704,458,19,0.261,0.326,0.443,0.691,1049,5.17,0.276,0.349,0.451


In [17]:
df_all = pd.DataFrame()

for i in range(1998, 2021):
    df = createAnnualDict3(i).sort_values(by='Team', ascending=True).reset_index().drop(columns = 'index')
    df['year'] = i
    col = df.pop("year")
    df.insert(1, col.name, col)
    df_all = pd.concat([df_all, df])

df_all.tail()

,Team,year,W,L,W-L%,RS,R/G,RA,RA/G,RD,AB,H,2B,3B,HR,RBI,BB,IBB,BA,OBP,SLG,DefEff,SO,ERA,OBA,OOBP,OSLG
25,St. Louis Cardinals,2020,30,28,0.517,240,4.14,229,3.95,11,1752,410,73,7,51,231,205,3,0.234,0.323,0.371,0.685,477,3.90,0.216,0.306,0.380
26,Tampa Bay Rays,2020,40,20,0.667,289,4.82,229,3.82,60,1975,470,105,12,80,274,243,9,0.238,0.328,0.425,0.675,608,3.56,0.238,0.305,0.391
27,Texas Rangers,2020,22,38,0.367,224,3.73,312,5.20,-88,1936,420,80,9,62,204,167,3,0.217,0.285,0.364,0.704,548,5.02,0.242,0.329,0.419
28,Toronto Blue Jays,2020,32,28,0.533,302,5.03,312,5.20,-10,2023,516,104,4,88,288,203,4,0.255,0.325,0.441,0.678,508,4.60,0.255,0.339,0.436
29,Washington Nationals,2020,26,34,0.433,293,4.88,301,5.02,-8,1968,519,112,12,66,279,192,16,0.264,0.336,0.433,0.667,451,5.09,0.271,0.346,0.466


In [18]:
# df_all.to_csv('mlb_team_stats_new.csv')

In [19]:
#get annual vectors for each team
def getAnnualTeamData2(teamName, year):
    
    annual_data = MLB_data[MLB_data['year'] == year]

    # number of games home and away
    gamesHome = annual_data[annual_data['Home_team_full'] == teamName] 
    gamesAway = annual_data[annual_data['Away_team_full'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)

    # runs per game
    totalRunsScored = gamesHome['score_home'].sum()
    totalRunsScored += gamesAway['score_away'].sum()
    totalRunsAllowed = gamesHome['score_away'].sum()
    totalRunsAllowed += gamesAway['score_home'].sum()
    
    #total AtBats
    totalAB = gamesHome['AB_home'].sum()
    totalAB += gamesAway['AB_away'].sum()
    totalABAllowed = gamesHome['AB_away'].sum()
    totalABAllowed += gamesAway['AB_home'].sum()

    #total Hits
    totalH = gamesHome['H_home'].sum()
    totalH += gamesAway['H_away'].sum()
    totalHAllowed = gamesHome['H_away'].sum()
    totalHAllowed += gamesAway['H_home'].sum()
    
    #total E
    totalE = gamesHome['E_home'].sum()
    totalE += gamesAway['E_away'].sum()
    
    #total 2B (doubles)
    total2B = gamesHome['2B_home'].sum()
    total2B += gamesAway['2B_away'].sum()
    total2BAllowed = gamesHome['2B_away'].sum()
    total2BAllowed += gamesAway['2B_home'].sum()
    
    #total 3B (triples)
    total3B = gamesHome['3B_home'].sum()
    total3B += gamesAway['3B_away'].sum()
    total3BAllowed = gamesHome['3B_away'].sum()
    total3BAllowed += gamesAway['3B_home'].sum()
    
    # total HR (home runs)
    totalHR = gamesHome['HR_home'].sum()
    totalHR += gamesAway['HR_away'].sum()
    totalHRAllowed = gamesHome['HR_away'].sum()
    totalHRAllowed += gamesAway['HR_home'].sum()

    # total RBI (run-batted in)
    totalRBI = gamesHome['RBI_home'].sum()
    totalRBI += gamesAway['RBI_away'].sum()

    # total SF (sac flies)
    totalSH = gamesHome['SH_home'].sum()
    totalSH += gamesAway['SH_away'].sum()
    
    # total SF (sac flies)
    totalSF = gamesHome['SF_home'].sum()
    totalSF += gamesAway['SF_away'].sum()
    totalSFAllowed = gamesHome['SF_away'].sum()
    totalSFAllowed += gamesAway['SF_home'].sum()

    # total HBP (hit by pitches)
    totalHBP = gamesHome['HBP_home'].sum()
    totalHBP += gamesAway['HBP_away'].sum()  
    totalHBPAllowed = gamesHome['HBP_away'].sum()
    totalHBPAllowed += gamesAway['HBP_home'].sum()
    
    # total BB
    totalBB = gamesHome['BB_home'].sum()
    totalBB += gamesAway['BB_away'].sum()
    totalBBAllowed = gamesHome['BB_away'].sum()
    totalBBAllowed += gamesAway['BB_home'].sum()
    
    # total IBB
    totalIBB = gamesHome['IBB_home'].sum()
    totalIBB += gamesAway['IBB_away'].sum()

    # total k
    totalSO = gamesHome['k_home'].sum()
    totalSO += gamesAway['k_away'].sum()

    # total teamER
    totalteamER = gamesHome['teamER_home'].sum()
    totalteamER += gamesAway['teamER_away'].sum()

    # total putouts
    totalputouts = gamesHome['putouts_home'].sum()
    totalputouts += gamesAway['putouts_away'].sum()

    #Batting stats
    gamesWonPercentage = numGamesWon / numGames
    RunsPerGame = totalRunsScored / numGames
    
    RunsDifference = totalRunsScored - totalRunsAllowed
    
    BA = totalH / totalAB
    OBP = (totalH + totalBB + totalHBP) / (totalAB + totalBB + totalHBP + totalSF)
    total1B = totalH - total2B - total3B - totalHR
    SLG = (1*total1B + 2*total2B + 3*total3B + 4*totalHR) / (totalAB)
    
    OBA = totalHAllowed / totalABAllowed
    OOBP = (totalHAllowed + totalBBAllowed + totalHBPAllowed) \
    / (totalABAllowed + totalBBAllowed + totalHBPAllowed + totalSFAllowed)
    total1BAllowed = totalHAllowed - total2BAllowed - total3BAllowed - totalHRAllowed
    OSLG = (1*total1BAllowed + 2*total2BAllowed + 3*total3BAllowed + 4*totalHRAllowed) / (totalABAllowed)
    
    #Defense stats
    totalPA = totalAB + totalBB + totalHBP + totalSF + totalSH
    RunsAllowedPerGame = totalRunsAllowed / numGames
    DefEff = 1 - ((totalH + totalE - totalHR)/(totalPA - totalBB - totalSO - totalHBP - totalHR))
    
    #Pitching stats
    ERA = (totalteamER / (totalputouts / 3)) * 9
    
    return [round(gamesWonPercentage, 3), round(RunsPerGame, 2), round(RunsAllowedPerGame, 2), int(RunsDifference), 
            int(totalAB), int(totalH), int(total2B), int(total3B), int(totalHR), 
            int(totalRBI), int(totalBB), int(totalIBB), 
            round(BA, 3), round(OBP, 3), round(SLG, 3),
            round(DefEff, 3), round(ERA, 2), round(OBA, 3), round(OOBP, 3), round(OSLG, 3)]


In [20]:
getAnnualTeamData2('Baltimore Orioles', 2019)

[0.333,
 4.5,
 6.06,
 -252,
 5596,
 1379,
 252,
 25,
 213,
 698,
 462,
 8,
 0.246,
 0.31,
 0.415,
 0.682,
 5.59,
 0.271,
 0.342,
 0.497]

In [21]:
import collections

def createAnnualDict2(year):
    annualDictionary = collections.defaultdict(list)
    teamList = MLB_data['Away_team_full'].unique()
    for team in teamList:
        team_vector = getAnnualTeamData2(team, year)
        annualDictionary[team] = team_vector
    return annualDictionary


In [22]:
def getTrainingData2(years):
    totalNumGames = 0
    for year in years:
        annual = MLB_data[MLB_data['year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData2('Baltimore Orioles', 2019)) #random team, to find dimensionality
    X = np.zeros(( totalNumGames, numFeatures))
    y = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict2(year)
        annual = MLB_data[MLB_data['year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['Home_team_full']
            a_team = row['Away_team_full']
            h_vector = team_vectors[h_team]
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                yTrainAnnual[counter] = 1
            # the opposite of the difference of the vectors should be 
            # a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        X[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        y[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return X, y

In [23]:
years = range(2013, 2021)
X, y = getTrainingData2(years)
np.save('X', X)
np.save('y', y)

In [24]:
X.shape

(17906, 20)

In [25]:
y.shape

(17906,)

In [26]:
print(X)

[[-0.243 -0.71   1.33  ...  0.024  0.034  0.058]
 [ 0.142  0.31  -0.75  ... -0.006 -0.011 -0.021]
 [-0.167 -1.12   1.01  ...  0.033  0.028  0.061]
 ...
 [-0.2    0.28   1.36  ...  0.041  0.043  0.077]
 [-0.116 -0.92   0.62  ...  0.003  0.007  0.012]
 [-0.116 -0.46  -0.3   ... -0.009 -0.02  -0.015]]


In [27]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
lm = linear_model.LogisticRegression()
model2 = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)
print(sum(predictions)/len(predictions))

(14324, 20) (14324,)
(3582, 20) (3582,)
0.57928531546622


In [29]:
def createGamePrediction(team1_vector, team2_vector):
    X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y, test_size = 0.2)
    lr_split = LogisticRegression()
    lr_model_split = lr_split.fit(X_train_split, y_train_split)
    diff = [a - b for a, b in zip(team1_vector, team2_vector)]
    diff = np.array(diff)
    diff = np.reshape(diff, (1, -1))
    predict_team = lr_model_split.predict(diff)
    predictions = lr_model_split.predict_proba(diff)
    return predict_team, predictions

In [30]:
team1_vector = getAnnualTeamData2('Los Angeles Dodgers', 2020)
team2_vector = getAnnualTeamData2('Tampa Bay Rays', 2020)
# print('Probability that LAD wins:', createGamePrediction(team1_vector, team2_vector, X, y))
# print('Probability that TB wins:', createGamePrediction(team2_vector, team1_vector, X, y))

In [31]:
createGamePrediction(team1_vector, team2_vector)

(array([1.]), array([[0.49784164, 0.50215836]]))

In [32]:
test_2021 = pd.read_excel(open('MLB Games 2021.xlsx', 'rb'), sheet_name='Full')
test_2021.tail()

,Unnamed: 0,Date,game_id,Year,Game_No,Day_of_week,Away_team,Away_team_full,Away_league,Away_conference,Away_gamenum,Home_team,Home_team_full,Home_league,Home_conference,Home_gamenum,Time_of_day,Note,Makeup_date,winning_team
2425,2425,2021-10-03,401229477,2021,0,Sun,NYM,New York Mets,NL,NL East,162,ATL,Atlanta Braves,NL,NL East,162,NaN,NaN,NaN,Atlanta Braves
2426,2426,2021-10-03,401229478,2021,0,Sun,OAK,Oakland Athletics,AL,AL West,162,HOU,Houston Astros,AL,AL West,162,NaN,NaN,NaN,Oakland Athletics
2427,2427,2021-10-03,401229479,2021,0,Sun,PHI,Philadelphia Phillies,NL,NL East,162,MIA,Miami Marlins,NL,NL East,162,NaN,NaN,NaN,Miami Marlins
2428,2428,2021-10-03,401229480,2021,0,Sun,SDP,San Diego Padres,NL,NL West,162,SFG,San Francisco Giants,NL,NL West,162,NaN,NaN,NaN,San Francisco Giants
2429,2429,2021-10-03,401229481,2021,0,Sun,TBR,Tampa Bay Rays,AL,AL East,162,NYY,New York Yankees,AL,AL East,162,NaN,NaN,NaN,New York Yankees


In [33]:
def formulatePredictions():
    probs = [[0 for x in range(4)] for x in range(len(test_2021.index))]
    for index, row in test_2021.iterrows():
        year = row['Year'] - 1
        team1_Name = row['Away_team_full']
        team2_Name = row['Home_team_full']
        team1_vector = getAnnualTeamData2(team1_Name, year)
        team2_vector = getAnnualTeamData2(team2_Name, year)
        prediction = createGamePrediction(team1_vector, team2_vector)
        
        predict_team = prediction[0]
        
        winner = None
        
        if predict_team == 0:
            winner = team1_Name
            loser = team2_Name
        else:
            loser = team1_Name
            winner = team2_Name
        
        probs[index][0] = prediction[0]
        probs[index][1] = prediction[1]
        probs[index][2] = winner
        probs[index][3] = loser
    probs = pd.np.array(probs)
    return probs

In [34]:
outcome2 = formulatePredictions()

In [35]:
draft2021 = test_2021.iloc[:, np.r_[1, 6:9, 11:14]]
draft2021['prob1'] = outcome2[:, 0]
draft2021['prob2'] = outcome2[:, 1]
draft2021['winning_team'] = outcome2[:, 2]
draft2021['losing_team'] = outcome2[:, 3]
draft2021

,Date,Away_team,Away_team_full,Away_league,Home_team,Home_team_full,Home_league,prob1,prob2,winning_team,losing_team
0,2021-04-01,ARI,Arizona D'Backs,NL,SDP,San Diego Padres,NL,[1.0],"[[0.4960585636238257, 0.5039414363761743]]",San Diego Padres,Arizona D'Backs
1,2021-04-01,ATL,Atlanta Braves,NL,PHI,Philadelphia Phillies,NL,[1.0],"[[0.49158407422522155, 0.5084159257747785]]",Philadelphia Phillies,Atlanta Braves
2,2021-04-01,BAL,Baltimore Orioles,AL,BOS,Boston Red Sox,AL,[1.0],"[[0.4983692310323097, 0.5016307689676903]]",Boston Red Sox,Baltimore Orioles
3,2021-04-01,CWS,Chicago White Sox,AL,LAA,Los Angeles Angels,AL,[0.0],"[[0.5028146340189243, 0.4971853659810756]]",Chicago White Sox,Los Angeles Angels
4,2021-04-01,CLE,Cleveland Indians,AL,DET,Detroit Tigers,AL,[1.0],"[[0.4923268353425141, 0.5076731646574859]]",Detroit Tigers,Cleveland Indians
...,...,...,...,...,...,...,...,...,...,...,...
2425,2021-10-03,NYM,New York Mets,NL,ATL,Atlanta Braves,NL,[1.0],"[[0.49302398702914063, 0.5069760129708594]]",Atlanta Braves,New York Mets
2426,2021-10-03,OAK,Oakland Athletics,AL,HOU,Houston Astros,AL,[1.0],"[[0.4951703626044627, 0.5048296373955373]]",Houston Astros,Oakland Athletics
2427,2021-10-03,PHI,Philadelphia Phillies,NL,MIA,Miami Marlins,NL,[0.0],"[[0.5011641709983017, 0.49883582900169826]]",Philadelphia Phillies,Miami Marlins
2428,2021-10-03,SDP,San Diego Padres,NL,SFG,San Francisco Giants,NL,[1.0],"[[0.4995114154047966, 0.5004885845952034]]",San Francisco Giants,San Diego Padres


In [36]:
team_after2013 = {
  "Arizona D'Backs": {"conference": 'NL West', "league": 'NL'},
  "Atlanta Braves": {"conference": 'NL East', "league": 'NL'},
  "Baltimore Orioles": {"conference": 'AL East', "league": 'AL'},
  "Boston Red Sox": {"conference": 'AL East', "league": 'AL'},
  "Chicago Cubs": {"conference": 'NL Central', "league": 'NL'},
  "Chicago White Sox": {"conference": 'AL Central', "league": 'AL'},
  "Cincinnati Reds": {"conference": 'NL Central', "league": 'NL'},
  "Cleveland Indians": {"conference": 'AL Central', "league": 'AL'},
  "Colorado Rockies": {"conference": 'NL West', "league": 'NL'},
  "Detroit Tigers": {"conference": 'AL Central', "league": 'AL'},
  "Houston Astros": {"conference": 'AL West', "league": 'AL'},
  "Kansas City Royals": {"conference": 'AL Central', "league": 'AL'},
  "Los Angeles Angels": {"conference": 'AL West', "league": 'AL'},
  "Los Angeles Dodgers": {"conference": 'NL West', "league": 'NL'},
  "Miami Marlins": {"conference": 'NL East', "league": 'NL'},
  "Milwaukee Brewers": {"conference": 'NL Central', "league": 'NL'},
  "Minnesota Twins": {"conference": 'AL Central', "league": 'AL'},
  "New York Mets": {"conference": 'NL East', "league": 'NL'},
  "New York Yankees": {"conference": 'AL East', "league": 'AL'},
  "Oakland Athletics": {"conference": 'AL West', "league": 'AL'},
  "Pittsburgh Pirates": {"conference": 'NL Central', "league": 'NL'},
  "Philadelphia Phillies": {"conference": 'NL East', "league": 'NL'},
  "San Diego Padres": {"conference": 'NL West', "league": 'NL'},
  "San Francisco Giants": {"conference": 'NL West', "league": 'NL'},
  "Seattle Mariners": {"conference": 'AL West', "league": 'AL'},
  "St. Louis Cardinals": {"conference": 'NL Central', "league": 'NL'},
  "Tampa Bay Rays": {"conference": 'AL East', "league": 'AL'},
  "Texas Rangers": {"conference": 'AL West', "league": 'AL'},
  "Toronto Blue Jays": {"conference": 'AL East', "league": 'AL'},
  "Washington Nationals": {"conference": 'NL East', "league": 'NL'}
}

def get_info(x, mode):
    return team_after2013[str(x)][mode]

In [37]:
winning = pd.DataFrame(draft2021.groupby(['winning_team'])['winning_team'].count())
losing = pd.DataFrame(draft2021.groupby(['losing_team'])['losing_team'].count())
tally = pd.concat([winning, losing], axis=1)
tally.reset_index(inplace=True)
tally = tally.rename(columns={"index": "team", "winning_team": "W", "losing_team": "L"})
tally['conference'] = tally['team'].map(lambda x: get_info(x, 'conference'))
tally['league'] = tally['team'].map(lambda x: get_info(x, 'league'))
tally = tally[['team', 'conference', 'league', 'W', 'L']]
tally = tally.sort_values(by=['conference', 'W'], ascending=[True, False])
tally

,team,conference,league,W,L
7,Cleveland Indians,AL Central,AL,88,74
11,Kansas City Royals,AL Central,AL,88,74
5,Chicago White Sox,AL Central,AL,77,85
9,Detroit Tigers,AL Central,AL,77,85
16,Minnesota Twins,AL Central,AL,74,88
3,Boston Red Sox,AL East,AL,83,79
28,Toronto Blue Jays,AL East,AL,82,80
18,New York Yankees,AL East,AL,81,81
2,Baltimore Orioles,AL East,AL,80,82
26,Tampa Bay Rays,AL East,AL,72,90


In [38]:
# draft2021.to_csv('predicted_2021_2.csv')